In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import importlib
import os, sys
sys.path.append(os.path.abspath('../../visualization'))
from plotters import plot_gp, plot_gpkf, plot_gpkf_space_time
from kernel import KernelFactory, Kernel, kernelFunction, kernelSampled

from sklearn import preprocessing

from params import Params
from gpkf import Gpkf
from generateSyntheticData import generateSyntheticData
#from plotters import plot_gp, plot_gp_example, plot_residuals

In [2]:
df = pd.read_csv('../../data/mergedData.csv')
df.set_index('Timestamp', inplace=True)
df.index = pd.to_datetime(df.index)
df.index = np.around(df.index.to_julian_date() * 24 * 60)
df.index = df.index - df.index[0]

In [3]:
center_df = df[[col for col in df.columns if 'Pos1' in col and 'DO' in col]] #.dropna(axis = 0, how = 'all')
#center_df = df['Pos1m3DO'].dropna(axis=0)
#center_df

In [4]:
def start_train(df):
    locations = np.array([3,6,9,12]).conj().T
    params = Params(df, locations, 'fornes')
    model = Gpkf(params)

    #F, Y, noiseVar = generateSyntheticData(params.data)
    print('Formatting data')
    
    F = df.loc[params.data['timeInstants']].to_numpy().T
    Y = df.to_numpy().T[params.data['spaceLocsMeasIdx']]
    Y_mean = np.nanmean(Y, axis=0)
    Y_std = np.nanstd(Y, axis=0)
    Y = (Y - Y_mean) / Y_std
    
    noiseVar = (params.data['noiseStd'] * np.abs(Y))**2
    noiseVar[np.isnan(noiseVar)] = np.inf
    noiseVar[Y==0] = params.data['noiseStd']**2
    
    #F, Y, noiseVar = generateSyntheticData(params.data)
    #Y_mean = Y.mean(axis=0)
    #Y = (Y - Y.mean(axis=0)) / Y.std(axis=0)
    
    # Optimize parameters
    print('Optimizing parameters (log-marginal)')
    model.optimize(Y, noiseVar, params.optimizer_restarts)
    
    # GPKF estimate
    print('GPKF estimation')
    posteriorMean, posteriorCov, logMarginal = model.estimation(Y, noiseVar)

    #GPKF prediction
    print('GPKF prediction')
    predictedMean, predictedCov = model.prediction(Y, noiseVar)
    
    #plot_gpkf(params.data['spaceLocsMeas'], params.data['spaceLocsMeasIdx'], F[:,-1][params.data['spaceLocsMeasIdx']], posteriorMean[:,-1], posteriorCov[:,:,-1])
    plot_gpkf(params.data['spaceLocsPred'], [3,6,9,12], F[:,-1], predictedMean[:,-1] + Y_mean[-1], predictedCov[:,:,-1])
    #plot_gpkf(np.arange(0, params.data['spaceLocsPredIdx'].size), np.arange(0,params.data['spaceLocsPredIdx'].size), F[:,-1][params.data['spaceLocsPredIdx']], predictedMean[:,-1], predictedCov[:,:,-1])

    return model

In [5]:
#model = start_train(center_df)
depthLocations = np.array([3,6,9,12]).conj().T
params = Params(center_df, depthLocations, 'fornes')

#F, Y, noiseVar = generateSyntheticData(params.data)
print('Formatting data')

F = center_df.loc[params.data['timeInstants']].to_numpy().T
Y = center_df.to_numpy().T[params.data['spaceLocsMeasIdx']]

model = Gpkf(params, Y, normalize_y = True, n_restarts = 0)

Using fornes dataset
Formatting data


TypeError: __init__() missing 2 required positional arguments: 'kernel_time' and 'kernel_space'

In [ ]:
# Optimize parameters
print('Optimizing parameters (log-marginal)')
#model.optimize()
#Y_mean = np.nanmean(Y)
#Y_mean

In [ ]:
# GPKF estimate
print('GPKF estimation')
posteriorMean, posteriorCov, logMarginal = model.estimation(prediction= False)

#GPKF prediction
print('GPKF prediction')
predictedMean, predictedCov = model.prediction()

In [ ]:
print(model)

In [ ]:
plot_gpkf_space_time(params, F, posteriorMean, posteriorCov, predictedMean, predictedCov, 1000, 7)
plt.savefig('test'+'.png')

In [ ]:
"""samples = np.random.multivariate_normal(posteriorMean[0,:].ravel(), posteriorCov[0,:,:], 1)
plot_gpkf(params.data['TimeInstants'], [], [], posteriorMean[0, :], posteriorCov[0,0,:], samples=samples)"""

# 3D

In [6]:
df_do = df[[col for col in df.columns if 'DO' in col]] #.dropna(axis = 0, how = 'all')
locations = df_do.iloc[0]
locations = locations.astype('object')
center = 30
for col in locations.index:
    depth = int(re.search('m\d+', col).group()[1:])  # Get sensor depth
    if 'Pos1' in col:
        locations[col] = np.array([center, depth, center])
    elif 'Pos4' in col:
        locations[col] = np.array([center +21, depth, center])
    elif 'Pos5' in col:
        locations[col] = np.array([center, depth, center+21])
    elif 'Pos6' in col:
        locations[col] = np.array([center - 21, depth, center])
    elif 'Pos7' in col:
        locations[col] = np.array([center, depth, center-21])
    elif 'Pos12' in col:
        locations[col] = np.array([center, depth, center])
    elif 'Pos21' in col:
        locations[col] = np.array([center-20, depth, center])
    elif 'Pos22' in col:
        locations[col] = np.array([center-27, depth, center])
    elif 'Pos23' in col:
        locations[col] = np.array([center-21, depth, center])
    elif 'Pos31' in col:
        locations[col] = np.array([center + 2, depth, center + 20])
    elif 'Pos32' in col:
        locations[col] = np.array([center, depth, center+27])
    elif col == 'Outsidem3DO':
        locations[col] = np.array([center-30, depth, center])
    elif col =='SouthOutsidem3DO':
        locations[col] = np.array([center, depth, center+27])
    elif col =='Northm3DO':
        locations[col] = np.array([center, depth, center+19])
    elif col =='Southm3DO':
        locations[col] = np.array([center, depth, center-19])
        
# Symmetric standardization of location inputs
for feature in locations.values.size:
    mean = locations.values[:,feature].mean(axis=0)
    std = locations.values[:,feature]
    

locations

In [27]:
params = Params(df_do, locations.values, 'fornes')

print('Formatting data')

F = df_do.loc[params.data['timeInstants']].to_numpy().T
Y = df_do.to_numpy().T[params.data['spaceLocsMeasIdx']]

print('Initializing Model')
# create time kernel
factory = KernelFactory()
kernel_time = factory.get_kernel('matern32', {'scale' : 0.6, 'std': 1.0, 'frequency':750}) # 0.6 and 0.0001

# create space kernel
kernel_space = factory.get_kernel('gaussian', {'scale' : 0.5 , 'std': 1.0, 'frequency':750})
model = Gpkf(params, Y, kernel_time, kernel_space, normalize_y = True, n_restarts = 0)

Using fornes dataset
Formatting data
Initializing Model


In [28]:
model.optimize()

126562.76859731376
126562.76809348738
126562.7687290395
126562.76859731376
126562.7687290395
126562.76863847022
6051011.767378534
6051011.767378534
6051011.767378534
6051011.767378534
6051011.767378534
6051011.767378534
6051001.709380532
6051001.709380533
6051001.70938025
6051001.709380532
6051001.709380247
6051001.709380533
6050842.872408518
6050842.872408513
6050842.872407478
6050842.872408518
6050842.8724074755
6050842.872408551
6050554.364521252
6050554.36452125
6050554.364517042
6050554.364521252
6050554.3645170545
6050554.364521997
6048511.4494889155
6048511.449489004
6048511.449463004
6048511.4494889155
6048511.449463003
6048511.449494307
6013792.732334381
6013792.732340954
6013792.731824327
6013792.732334381
6013792.7318243245
6013792.732373987
4274294.370187606
4274294.371732484
4274294.3533240175
4274294.370187606
4274294.353324016
4274294.371858236
156536.1244078698
156536.1263133805
156536.12328648515
156536.1244078698
156536.12328648518
156536.12517824644
124645.5411800358

KeyboardInterrupt: 

In [29]:
# GPKF estimate
print('GPKF estimation')
posteriorMean, posteriorCov, logMarginal = model.estimation(prediction= False)

#GPKF prediction
print('GPKF prediction')
predictedMean, predictedCov = model.prediction()

print(model)

GPKF estimation
GPKF prediction
[56952.83246972]
Time kernel: {'scale': 12.23889302164178, 'std': 23.965945948218565, 'frequency': 750.00000001}
Space kernel: {'scale': 14.55405306571424, 'std': 1e-05, 'frequency': 750}


In [ ]:
X = np.ones((60, 3)) * 30
X[:,0] = np.linspace(0,60, 60)
X[:,1] = 3
number_of_functions = 1

sig = kernel_space.sampled(X, X)
mu = np.zeros(60) * 10
print(posteriorMean.shape)
samp = np.random.multivariate_normal(posteriorMean[:,1000].ravel(), posteriorCov[:,:,1000], number_of_functions)

ax = sns.heatmap(samp, xticklabels=1, yticklabels = 1, cmap='viridis', cbar_kws=dict(pad=0.01, aspect=10))

plt.figure(figsize=(6, 4), dpi=100)
for i in range(number_of_functions):
    plt.plot(X[:,0], samp[i], linestyle='-', marker='o', markersize=3)
plt.axhline(y=0.0, color='black', linestyle='-')
plt.xlabel('$x$', fontsize=13)
plt.ylabel('$y = f(x)$', fontsize=13)

In [ ]:
posteriorCov[:,:,1000].ravel().shape
posteriorMean[:,1000]

In [ ]:
m3Indexes = np.array([key for key, value in enumerate(locations.values) if value[1]==3])
m3Pos = np.array([value[[0,2]] for key, value in enumerate(locations.values) if value[1]==3])
vals = np.zeros(shape=(11,11))
for key,value in enumerate(params.data['spaceLocsPred']):
    vals[int(value[0]/6),int(value[2]/6)] = predictedMean[key,1000]
print(predictedMean[-2,1000])

ax = sns.heatmap(vals, xticklabels=1, yticklabels = 1, cmap='viridis', cbar_kws=dict(pad=0.01, aspect=10))
ax.invert_yaxis()

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
xx, yy = np.meshgrid(np.arange(0,11), np.arange(0,11))
ax.plot_surface(xx, yy, vals, cmap='viridis',
                       linewidth=0, antialiased=False)

In [ ]:
predictedMean[:,1000]
#params.data['spaceLocsMeas']